##Setup libraries, dataset path and movie genres

In [239]:
import pandas as pd
import os
dataset_path = "../dataset"
movie_genres = ['action', 'adventure', 'animation', 'biography', 'crime', 'family', 'fantasy', 
          'film-noir', 'history', 'horror', 'mystery', 'romance', 'scifi', 'sports', 
          'thriller', 'war']

##Read each file and store dataframes separately

In [240]:
# Define a dictionary to store dataframes for each genre
movies_by_genre = {}

# Update the dictionary
for file in os.listdir(dataset_path):
    for genre in movie_genres:
        if genre in file:
            if genre not in movies_by_genre:
                movies_by_genre[genre] = pd.read_csv(os.path.join(dataset_path, file))
            else:
                movies_by_genre[genre] = pd.concat([movies_by_genre[genre], pd.read_csv(os.path.join(dataset_path, file))])

# Print the number of rows and columns in each genre dataframe without any data quality checks
print("genre, number of rows, number of columns")
for genre, movies_df in movies_by_genre.items():
    print(genre, movies_df.shape)

genre, number of rows, number of columns
action (52452, 14)
adventure (25664, 14)
animation (8419, 14)
biography (8289, 14)
crime (35852, 14)
family (17095, 14)
fantasy (17163, 14)
film-noir (986, 14)
history (8996, 14)
horror (36682, 14)
mystery (18960, 14)
romance (52617, 14)
scifi (16557, 14)
sports (5292, 14)
thriller (53365, 14)
war (9911, 14)


##Perform data quality checks

In [241]:
# Dimensionality reduction
# for each dataframe in the dictionary, rename the columns and select only the columns we need
for genre, movies_df in movies_by_genre.items():
    movies_df.rename(columns={'gross(in $)': 'gross'}, inplace=True)
    movies_df.rename(columns={'movie_name': 'movie'}, inplace=True)
    movies_by_genre[genre] = movies_df[['movie', 'year', 'runtime', 'rating', 'director', 'star', 'votes', 'gross']]

# print the columns selected for each genre
for genre, movies_df in movies_by_genre.items():
    print(genre, movies_df.columns.values)

action ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
adventure ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
animation ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
biography ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
crime ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
family ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
fantasy ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
film-noir ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
history ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
horror ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
mystery ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
romance ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
scifi ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'v

##The study of trends over the years, specifically the popularity of genres
For this case study we will consider columns that can be used to visualize the trends in the movie industry over the years. We will consider the following columns: movie, year, runtime, rating, director, star. We will also consider the following columns for further analysis: gross

In [242]:
popularity_of_genres = {}
# remove null values 
for genre, movies_df in movies_by_genre.items():
    popularity_of_genres[genre] = movies_df[movies_df['year'].notnull()]

# number of rows containing null values for each genre
print("null values for each genre")
print("----------------------------------")
for genre, movies_df in movies_by_genre.items():
    print(genre, movies_df.shape[0] - popularity_of_genres[genre].shape[0])



null values for each genre
----------------------------------
action 8259
adventure 3689
animation 1369
biography 2005
crime 3563
family 1631
fantasy 2814
film-noir 0
history 1295
horror 7591
mystery 2004
romance 3437
scifi 4018
sports 575
thriller 10137
war 861


In [250]:
# number of rows for popularity_of_genres
print("number of rows for popularity_of_genres")
print("----------------------------------")
for genre, movies_df in movies_by_genre.items():
    print(genre, popularity_of_genres[genre].shape[0])

number of rows for popularity_of_genres
----------------------------------
action 42284
adventure 21371
animation 6868
biography 5973
crime 31501
family 15197
fantasy 13852
film-noir 986
history 7494
horror 26920
mystery 16497
romance 48512
scifi 11399
sports 4599
thriller 39896
war 8891


##Noise analysis

In [252]:
# try to normalize the year column values to a 4 digit year format
for genre, movies_df in movies_by_genre.items():
    movies_df['year'] = movies_df['year'].astype(str).str[:4]

# remove values that are not in the years 1950-2023
for genre, movies_df in movies_by_genre.items():
    movies_by_genre[genre] = movies_df[(movies_df['year'] >= '1950') & (movies_df['year'] <= '2023')]

# check if there are noises in the year column
# by looking if there is a value outside of the years 1950-2023
for genre, movies_df in movies_by_genre.items():
    print(genre, movies_df[(movies_df['year'] < '1950') | (movies_df['year'] > '2023')].shape[0])



action ['2022' '2023' '2018' '1986' '2009' '2012' '2021' '2001' '2008' '2010'
 '2000' '2019' '2014' '2017' '1999' '1981' '1996' '2020' '1995' '1993'
 '1988' '2015' '2003' '2013' '1994' '1989' '1977' '2016' '2002' '2011'
 '1982' '1984' '1991' '2007' '1997' '2006' '2004' '2005' '1987' '1967'
 '1963' '1964' '1979' '1965' '1980' '1998' '1983' '1990' '1962' '1992'
 '1985' '1978' '1960' '1976' '1954' '1971' '1959' '1973' '1969' '1974'
 '1968' '1972' '1975' '1952' '1966' '1961' '1953' '1970' '1958' '1957'
 '1950' '1955' '1956' '1951']
adventure ['2022' '2023' '2018' '2009' '2001' '2014' '2012' '2021' '2010' '2000'
 '2019' '2011' '1981' '1996' '2005' '1999' '1985' '1993' '2017' '1988'
 '1990' '2003' '1980' '2013' '1987' '2015' '2004' '1977' '1995' '2002'
 '2016' '1972' '2020' '1989' '1986' '2007' '1994' '1997' '2006' '1968'
 '1984' '1966' '1991' '1967' '1975' '2008' '1963' '1964' '1965' '1998'
 '1983' '1982' '1979' '1962' '1992' '1978' '1960' '1976' '1959' '1957'
 '1971' '1973' '1956' '1974' '